In [1]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import *
import sklearn.metrics as metrics
import pickle

2024-08-18 22:23:40.250138: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-18 22:23:40.251781: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-18 22:23:40.260543: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-18 22:23:40.287924: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 22:23:40.332946: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [3]:
tf.__version__

'2.17.0'

In [4]:
data = pd.read_csv('../asset/smart_grid_stability_augmented.csv')

In [5]:
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [6]:
data.shape

(60000, 14)

In [7]:
mapping = {'unstable': 0, 'stable': 1}
data['stabf'] = data['stabf'].map(mapping)

In [8]:
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,0
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,1
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,0
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,0
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,0


In [9]:
data = data.drop('stab', axis=1)

In [10]:
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,1
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0


In [11]:
X = data.drop('stabf', axis=1)
y = data['stabf']

In [12]:
X.shape, y.shape

((60000, 12), (60000,))

In [13]:
X.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923


In [14]:
y.head()

0    0
1    1
2    0
3    0
4    0
Name: stabf, dtype: int64

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [16]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((42000, 12), (18000, 12), (42000,), (18000,))

In [17]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [18]:
X_val.shape, X_test.shape, y_val.shape, y_test.shape

((9000, 12), (9000, 12), (9000,), (9000,))

In [19]:
print("Percentage of stable grid in Training set: ", (y_train[y_train == 1].count() / y_train.count()) * 100, "%")
print("Percentage of stable grid in Testing set: ", (y_test[y_test == 1].count() / y_test.count()) * 100, "%")
print("Percentage of stable grid in Validation set: ", (y_val[y_val == 1].count() / y_val.count()) * 100, "%")

Percentage of stable grid in Training set:  36.18571428571429 %
Percentage of stable grid in Testing set:  36.922222222222224 %
Percentage of stable grid in Validation set:  35.544444444444444 %


In [20]:
scalar = StandardScaler()
X_train_scaled = scalar.fit_transform(X_train)
X_val_scaled = scalar.transform(X_val)
X_test_scaled = scalar.transform(X_test)

In [21]:
X_train.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
918,7.846974,8.877354,5.843796,5.895582,3.983750,-1.618278,-0.728624,-1.636848,0.932917,0.581647,0.640909,0.305371
17141,3.007313,8.684800,6.074018,0.858635,4.230058,-0.773622,-1.937173,-1.519263,0.924423,0.941520,0.793961,0.133047
15558,7.992806,4.957976,7.922754,7.774422,3.566676,-0.682930,-1.272413,-1.611333,0.942250,0.818530,0.973506,0.400792
27327,2.244237,7.194479,8.293343,7.809539,2.430400,-0.517186,-0.662992,-1.250222,0.363491,0.136525,0.082320,0.773569
11606,4.471082,7.895908,7.413906,1.128001,3.221704,-1.950386,-0.598811,-0.672507,0.142971,0.820521,0.531451,0.811377


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [23]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(12,)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/home/aniket/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1724000039.821368   22933 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-18 22:23:59.822400: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.

In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,033 (47.00 KB)

 Trainable params: 12,033 (47.00 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_val_scaled, y_val))

Epoch 1/100
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8179 - loss: 0.3848 - val_accuracy: 0.9370 - val_loss: 0.1636
Epoch 2/100
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9157 - loss: 0.1988 - val_accuracy: 0.9534 - val_loss: 0.1216
Epoch 3/100
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9284 - loss: 0.1687 - val_accuracy: 0.9631 - val_loss: 0.1001
Epoch 4/100
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9353 - loss: 0.1499 - val_accuracy: 0.9649 - val_loss: 0.0946
Epoch 5/100
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9409 - loss: 0.1409 - val_accuracy: 0.9676 - val_loss: 0.0914
Epoch 6/100
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9439 - loss: 0.1329 - val_accuracy: 0.9690 - val_loss: 0.0877
Epoch 7/100
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9484 - loss: 0.1251 - val_accuracy: 0.9664 - val_loss: 0.0925
Epoch 8/100
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9492 - loss: 0

In [26]:
accuracy = model.evaluate(X_test_scaled, y_test)

282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9776 - loss: 0.0569


In [27]:
accuracy

[0.05941171571612358, 0.9778888821601868]

In [28]:
X_test.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
54482,6.547591,6.438867,6.593892,4.357273,3.373254,-0.682392,-1.115608,-1.575254,0.572289,0.362432,0.520139,0.443506
43834,9.433586,9.077816,2.227302,2.609427,4.334689,-1.731135,-1.373889,-1.229666,0.505025,0.791719,0.057549,0.994605
45667,1.402361,6.245803,3.208440,5.230461,4.499151,-1.377323,-1.224031,-1.897797,0.191743,0.457570,0.485068,0.056872
52890,4.714897,3.342714,1.125952,7.394299,4.918754,-1.038290,-1.926761,-1.953703,0.967877,0.686775,0.315758,0.339957
41358,4.454121,6.826867,3.633670,9.850580,2.783338,-1.178827,-1.037554,-0.566957,0.672379,0.127205,0.681394,0.757251


In [29]:
X.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923


In [30]:
tau1_val=9.3
tau2_val=4.9 
tau3_val=3.04 
tau4_val=1.36
p1_val=5.06
p2_val=-1.94
p3_val=-1.87
p4_val=-1.25
g1_val=0.41
g2_val=0.86
g3_val=0.56
g4_val=0.78

In [31]:
# test model with new data
new_data = [[tau1_val, tau2_val, tau3_val, tau4_val, p1_val, p2_val, p3_val, p4_val, g1_val, g2_val, g3_val, g4_val]]
new_data_scaled = scalar.transform(new_data)
prediction = model.predict(new_data_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step


/home/aniket/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [32]:
prediction

array([[0.8984674]], dtype=float32)

In [33]:
while True:
    tau1_val = float(input("Enter tau1: "))
    tau2_val = float(input("Enter tau2: "))
    tau3_val = float(input("Enter tau3: "))
    tau4_val = float(input("Enter tau4: "))
    p1_val = float(input("Enter p1: "))
    p2_val = float(input("Enter p2: "))
    p3_val = float(input("Enter p3: "))
    p4_val = float(input("Enter p4: "))
    g1_val = float(input("Enter g1: "))
    g2_val = float(input("Enter g2: "))
    g3_val = float(input("Enter g3: "))
    g4_val = float(input("Enter g4: "))

    new_data = [[tau1_val, tau2_val, tau3_val, tau4_val, p1_val, p2_val, p3_val, p4_val, g1_val, g2_val, g3_val, g4_val]]
    new_data_scaled = scalar.transform(new_data)
    prediction = model.predict(new_data_scaled)
    print(prediction)

    if input("Do you want to continue? (y/n): ") == 'n':
        break

ValueError: could not convert string to float: ''

In [34]:
# what will be the threshold value for the model?
y_pred = model.predict(X_test_scaled)
y_pred = (y_pred > 0.6)
cm = confusion_matrix(y_test, y_pred)
cm

282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([[5639,   38],
       [ 206, 3117]])

In [35]:
y_test.value_counts()

stabf
0    5677
1    3323
Name: count, dtype: int64

In [36]:
from tensorflow import lite

In [37]:
converter = lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("smart_grid_stability_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpp_ekzz0u/assets


INFO:tensorflow:Assets written to: /tmp/tmpp_ekzz0u/assets


Saved artifact at '/tmp/tmpp_ekzz0u'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 12), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  126546118922448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126546082740560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126546082737872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126546079712080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126546079715344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126546079717072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126546079716880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126546079717456: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1724002581.377479   22933 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1724002581.377544   22933 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-08-18 23:06:21.378190: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpp_ekzz0u
2024-08-18 23:06:21.379666: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-08-18 23:06:21.379696: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpp_ekzz0u
2024-08-18 23:06:21.389658: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-08-18 23:06:21.391579: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-08-18 23:06:21.460558: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpp_ekzz0u
2024-08-18 23:06:21.482717: I tensorflow/cc/saved_model/loader.cc

50556